In [1]:

from src.analysis.aggregate_trial_results import aggregate_trial_results
from src.analysis.clean_balance_filter import (
    print_rm_anova_summary,
    clean_balance_filter,
    run_repeated_measures_anova
)
from src.analysis.df_formatting import (
    print_nice_dataframe,
    make_long_df
)
# Create summary stats using default results folder path
aggregate_trial_results = aggregate_trial_results()

# Prepare data for analysis
cleaned_df_31 = clean_balance_filter(aggregate_trial_results, 'llama_31', verbose=True)

# long_df_31 = make_long_df(cleaned_df_31)

# # Print the updated DataFrame
# print_nice_dataframe(long_df_31)

# # Run the repeated measures ANOVA
# anova_results_31 = run_repeated_measures_anova(long_df_31)

Aggregated trial results saved to /workspaces/coordination/pipeline/4_analysis/trial_results_aggregated.csv

Data after filtering by model family and reasoning conditions:
Displaying first 20 rows (total: 189):

+------------------------------------------------------------+--------------+---------------+--------------+--------------------+------------------+----------------------+-------------------+--------------------+----------------------+-----------------------+---------------------+----------------------+----------------------+-----------------------+--------------------+------------------+---------------+-------------------+----------------+-------------------+------------------+-------------------+------------------------+-------------------+---------------------------+--------------------------+----------------------------+--------------------------+-------------------------+---------------------------+
| file_name                                                  | model_name 

In [ ]:
#overview
# print_rm_anova_summary(anova_results_31)

In [ ]:
from src.analysis.visualization import (
    plot_model_comparison,
    create_comprehensive_analysis,
    plot_control_reasoning_bars
)
from src.analysis.df_formatting import print_nice_dataframe
# create_comprehensive_analysis(long_df_31)
# plot_model_comparison(long_df_31)
# print_nice_dataframe(long_df_31)
# plot_control_reasoning_bars(long_df_31)